# Training VAE for RGB Images

### Imports

In [3]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.loaders import FeaturesExtendedDataset

from models import FC_VAE
from train_vae import train, evaluate


### SETUP

In [4]:
BATCH_SIZE = 16
EPOCHS = 50
LR = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 15
GAMMA = 0.1

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")

features_file = "saved_features/saved_feat_I3D_25_dense_D1"

### TRAINING

In [3]:
train_dataset = FeaturesExtendedDataset(features_file,'train')
train_loader_rgb = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)


model = FC_VAE(dim_input=1024, nz=64, n_hidden= 512,device='cpu')
model.to(DEVICE)
print(f'Initial model device: {model.device}')

# Create Optimizer & Scheduler objects
optimizer = Adam(model.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

train(model, optimizer, EPOCHS, DEVICE, train_loader_rgb, train_loader_rgb, BATCH_SIZE, scheduler)

torch.save(model.state_dict(), f'./saved_models/VAE_RGB/final_VAE_RGB_epoch_{EPOCHS}.pth')


Initial model device: cpu


482it [00:20, 23.39it/s]
2024-06-12 18:57:53 LOG INFO 	Epoch, 1, 	Average Loss: , 0.00391683539447303
482it [00:23, 20.88it/s]
2024-06-12 18:58:46 LOG INFO 	Epoch, 2, 	Average Loss: , 0.003042558766707871
482it [00:21, 22.12it/s]
2024-06-12 18:59:38 LOG INFO 	Epoch, 3, 	Average Loss: , 0.0027793091276501793
482it [00:22, 21.87it/s]
2024-06-12 19:00:32 LOG INFO 	Epoch, 4, 	Average Loss: , 0.0026304349332288866
482it [00:23, 20.20it/s]
2024-06-12 19:01:25 LOG INFO 	Epoch, 5, 	Average Loss: , 0.002539346013711901
482it [00:23, 20.10it/s]
2024-06-12 19:02:23 LOG INFO 	Epoch, 6, 	Average Loss: , 0.0024693841361508487
482it [00:25, 19.07it/s]
2024-06-12 19:03:21 LOG INFO 	Epoch, 7, 	Average Loss: , 0.0024127200983557655
482it [00:26, 18.09it/s]
2024-06-12 19:04:23 LOG INFO 	Epoch, 8, 	Average Loss: , 0.00235726615305595
482it [00:27, 17.43it/s]
2024-06-12 19:05:23 LOG INFO 	Epoch, 9, 	Average Loss: , 0.0023171110245691546
482it [00:29, 16.23it/s]
2024-06-12 19:06:27 LOG INFO 	Epoch, 10, 	Ave

### EVALUATION

In [6]:
train_dataset = FeaturesExtendedDataset(features_file,'train')
test_dataset = FeaturesExtendedDataset(features_file,'test')
train_loader_rgb = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=4, drop_last=True)
test_loader_rgb = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4, drop_last=True)

model = FC_VAE(dim_input=1024, nz=64, n_hidden= 512, device='cpu')
model.to(DEVICE)
model.load_state_dict(torch.load(f'./saved_models/VAE_RGB/final_VAE_RGB_epoch_50.pth'))

reconstructed, originals = evaluate(model, DEVICE, train_loader_rgb,train_loader_rgb)
reconstructed2, originals2 = evaluate(model, DEVICE, test_loader_rgb,test_loader_rgb)

#2024-06-12 18:20:00 LOG INFO 	Epoch, 30, 	Average Loss: , 0.0020544183812745056
#Test Loss: 0.0001

7715it [00:38, 199.20it/s]


Test Loss: 0.0294


2175it [00:14, 149.40it/s]


Test Loss: 0.0294


In [10]:
import torch.nn as nn

n = 16
print(reconstructed2[n])
print(originals2[n])
mse_loss = nn.MSELoss()

import torch
import torch.nn as nn

# I due tensori tra cui calcolare la NMSE
reconstructed_data = torch.tensor(reconstructed2[n])  # Tensor dei dati ricostruiti
original_data = torch.tensor(originals2[n])       # Tensor dei dati originali

# Calcola la norma del vettore dei dati effettivi
norm_original_data = torch.norm(original_data)

# Definisci la funzione di perdita MSE
mse_loss = nn.MSELoss()

# Calcola l'errore quadratico medio
mse = mse_loss(reconstructed_data, original_data)

# Calcola l'errore quadratico medio normalizzato
nmse = mse / (norm_original_data ** 2)

print("Normalized Mean Squared Error:", nmse.item())
print("norm_original_data:", norm_original_data.item())



# Calcola l'errore assoluto
absolute_error = torch.abs(reconstructed_data - original_data)

# Calcola il MAE
mae = torch.mean(absolute_error)
print("MSE:", mse.item())
print("Mean Absolute Error (MAE):", mae.item())



[0.1617706  0.18080583 0.27730548 ... 0.26700526 0.4048736  0.2513174 ]
[0.25173348 0.06175843 0.6073024  ... 0.6841187  0.21441153 0.4415941 ]
Normalized Mean Squared Error: 0.0003446452901698649
norm_original_data: 8.725092887878418
MSE: 0.026236895471811295
Mean Absolute Error (MAE): 0.11557242274284363


# Training VAE for EMG Signals

In [11]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.loaders import FeaturesExtendedEMGDataset

from models import FC_VAE
from train_vae import train_emg, evaluate_emg
import pandas as pd
import numpy as np

### SETUP

In [12]:
BATCH_SIZE = 16
EPOCHS = 50
LR = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 15
GAMMA = 0.1

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")

LSTM_features_file_train = "saved_features/NEW_EMG_Emb_LSTM_25_dense_D1_train.pkl"
LSTM_features_file_test = "saved_features/NEW_EMG_Emb_LSTM_25_dense_D1_test.pkl"
STAT_features_file_train = "saved_features/EMG_Emb_Stat_25_dense_D1_train.pkl"
STAT_features_file_test = "saved_features/EMG_Emb_Stat_25_dense_D1_test.pkl"

### TRAINING

In [13]:
train_dataset = FeaturesExtendedEMGDataset(LSTM_features_file_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)

for i in train_loader:
    print(i["features"].shape)
    break

torch.Size([16, 1024])


In [12]:
for (rgb_batch_idx, x) in enumerate(train_loader):
    print(x)
    break

{'features': tensor([[-6.6190e-02,  5.6530e-02,  1.2611e-01,  ..., -2.8958e-01,
          2.5173e-02, -9.0646e-02],
        [-2.2526e-01,  1.1517e-01, -4.2411e-02,  ..., -1.9744e-01,
          4.1075e-03,  1.7464e-02],
        [-8.8189e-02,  6.5948e-02,  1.6635e-01,  ..., -2.5759e-01,
         -1.8692e-03, -1.6630e-02],
        ...,
        [ 9.3071e-01,  7.3564e-02, -1.2945e-01,  ..., -2.4029e-01,
          1.4612e-01,  2.9741e-02],
        [ 8.8873e-01, -2.4171e-02, -1.5686e-01,  ..., -5.0851e-01,
          2.2111e-01,  3.0519e-02],
        [-3.5172e-01,  8.0837e-02, -1.6099e-02,  ...,  4.1486e-01,
         -2.2982e-04, -2.5483e-01]]), 'labels': tensor([[ 2],
        [ 2],
        [ 2],
        [ 2],
        [ 2],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [ 1],
        [10],
        [10],
        [10],
        [10],
        [10],
        [19]])}


In [14]:
model = FC_VAE(dim_input=1024, nz=64, n_hidden= 512,device='cpu')
model.to(DEVICE)
print(f'Initial model device: {model.device}')

# Create Optimizer & Scheduler objects
optimizer = Adam(model.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

train_emg(model, optimizer, EPOCHS, DEVICE, train_loader, train_loader, BATCH_SIZE, scheduler)

torch.save(model.state_dict(), f'./saved_models/VAE_EMG/final_VAE_EMG_epoch_{EPOCHS}.pth')

Initial model device: cpu


560it [00:25, 21.55it/s]
2024-06-12 20:03:06 LOG INFO 	Epoch, 1, 	Average Loss: , 0.0059517194549861315
560it [00:24, 23.25it/s]
2024-06-12 20:04:11 LOG INFO 	Epoch, 2, 	Average Loss: , 0.004231555360304408
560it [00:25, 22.28it/s]
2024-06-12 20:05:13 LOG INFO 	Epoch, 3, 	Average Loss: , 0.00353982230862486
560it [00:25, 22.33it/s]
2024-06-12 20:06:14 LOG INFO 	Epoch, 4, 	Average Loss: , 0.003006360943850683
560it [00:24, 23.20it/s]
2024-06-12 20:07:14 LOG INFO 	Epoch, 5, 	Average Loss: , 0.0028009914945309553
560it [00:25, 22.38it/s]
2024-06-12 20:08:15 LOG INFO 	Epoch, 6, 	Average Loss: , 0.0025645678230662946
560it [00:23, 23.66it/s]
2024-06-12 20:09:15 LOG INFO 	Epoch, 7, 	Average Loss: , 0.0024349310696508224
560it [00:25, 21.76it/s]
2024-06-12 20:10:20 LOG INFO 	Epoch, 8, 	Average Loss: , 0.0022490288185918693
560it [00:25, 21.60it/s]
2024-06-12 20:11:25 LOG INFO 	Epoch, 9, 	Average Loss: , 0.002094579841512587
560it [00:25, 22.21it/s]
2024-06-12 20:12:30 LOG INFO 	Epoch, 10, 	Av

### EVALUATION

In [14]:
train_dataset = FeaturesExtendedEMGDataset(LSTM_features_file_train)
test_dataset = FeaturesExtendedEMGDataset(LSTM_features_file_test)
train_loader_emg = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4, drop_last=True)
test_loader_emg = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=4, drop_last=True)

model = FC_VAE(dim_input=1024, nz=64, n_hidden= 512, device='cpu')
model.to(DEVICE)
model.load_state_dict(torch.load(f'./saved_models/VAE_EMG/final_VAE_EMG_epoch_50.pth'))

reconstructed, originals = evaluate_emg(model, DEVICE, train_loader_emg, train_loader_emg)
reconstructed2, originals2 = evaluate_emg(model, DEVICE, test_loader_emg, test_loader_emg)

8975it [00:51, 175.91it/s]


Test Loss: 0.0168


8975it [00:51, 174.47it/s]


Test Loss: 0.0168


In [18]:
import torch.nn as nn
import torch
n = 16
print(reconstructed2[n])
print(originals2[n])
mse_loss = nn.MSELoss()



# I due tensori tra cui calcolare la NMSE
reconstructed_data = torch.tensor(reconstructed2[n])  # Tensor dei dati ricostruiti
original_data = torch.tensor(originals2[n])       # Tensor dei dati originali

# Calcola la norma del vettore dei dati effettivi
norm_original_data = torch.norm(original_data)

# Definisci la funzione di perdita MSE
mse_loss = nn.MSELoss()

# Calcola l'errore quadratico medio
mse = mse_loss(reconstructed_data, original_data)

# Calcola l'errore quadratico medio normalizzato
nmse = mse / (norm_original_data ** 2)

print("Normalized Mean Squared Error:", nmse.item())
print("norm_original_data:", norm_original_data.item())



# Calcola l'errore assoluto
absolute_error = torch.abs(reconstructed_data - original_data)

# Calcola il MAE
mae = torch.mean(absolute_error)
print("MSE:", mse.item())
print("Mean Absolute Error (MAE):", mae.item())



[-0.0075567  -0.01152161  0.1278025  ... -0.09474144  0.02167922
 -0.07887158]
[-0.03953028 -0.03217608  0.11024177 ... -0.09489374  0.04725005
 -0.10085487]
Normalized Mean Squared Error: 0.0002937364624813199
norm_original_data: 2.5818302631378174
MSE: 0.0019580023363232613
Mean Absolute Error (MAE): 0.031950294971466064


# Fine Tuning Training [ RGB --> EMG ]

### IMPORTS

In [19]:
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.loaders import FeaturesExtendedDataset, FeaturesExtendedEMGDataset, ActionNetEmgRgbDataset
from models import I3D
from models import EMG_Feature_Extractor
from utils.args import args
from omegaconf import OmegaConf
import tqdm
import pickle


from models import FC_VAE, LSTM_Emb_Classifier, EMG_Feature_Extractor
from train_vae import train_tuning, evaluate_tuning

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")

In [29]:
# ---------------------- I3D ----------------------
# CONFIGURATION FOR I3D
conf_args = OmegaConf.load('configs/I3D_save_feat.yaml')
args = OmegaConf.merge(args, conf_args)

model_rgb = I3D(20, "RGB", args.models['RGB'], **args.models['RGB'].kwargs)
train_augmentations, test_augmentations = model_rgb.get_augmentation('RGB')
model_rgb.to("cpu")

# ---------------------- LSTM ----------------------
# Parametri del modello
input_dim = 16
hidden_dim = 300
embedding_dim = 256
output_dim = 20  # Definisci il numero di classi

model_emg = LSTM_Emb_Classifier(input_dim=input_dim, hidden_dim=hidden_dim, embedding_dim=embedding_dim, num_class=output_dim)
model_emg.load_state_dict(torch.load(f'./saved_models/LSTM_Emb_Classifier/NEW_final_LSTM_Emb_25_epoch_10.pth'))
model_emg.to(DEVICE)

# ---------------------- DATASET ----------------------
# DATASET 25 FRAMES PER CLIP AND 1 SAMPLE PER BATCH
num_frames = 25
num_clips = 1
batch_size = 1
dataset = ActionNetEmgRgbDataset('train', num_frames, num_clips, True, './action-net', 'action-net/saved_emg', "D:/ActionNetDataset/saved_RGB/frames", 2, train_augmentations)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=False)
len(dataset)


2024-06-12 21:16:52 LOG INFO Loading Kinetics weights I3D
2024-06-12 21:16:52 LOG INFO  * Skipping Logits weight for 'logits.conv3d.weight'
2024-06-12 21:16:52 LOG INFO  * Skipping Logits weight for 'logits.conv3d.bias'
c:\Users\iplov\miniconda3\envs\aml_ego\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


186

### FEATURE EXTRACTION ACTION NET s04

In [30]:
embeddings_rgb = []
embeddings_emg = []


model_rgb.train(False)
with torch.no_grad():
    for (idx, (emg,rgb,l)) in enumerate(data_loader):
        e = emg.reshape(batch_size, num_clips, num_frames, -1)  #(num_batch, num_clips, num_frames, num_features)
        # print(e)                  # torch.Size([1, 1, 25, 16])
        emg_input = e[0].float() # torch.Size([25, 16])
        emg_input = emg_input.to(DEVICE)

        batch, _, height, width = rgb.shape
        rgb_reshape = rgb.reshape(1, num_clips, num_frames, -1, height, width)
        rgb_permute = rgb_reshape.permute(1, 0, 3, 2, 4, 5)
        rgb_input = rgb_permute[0].to('cpu')    # CLIP
        # print(rgb_permute.shape)   torch.Size([1, 1, 3 (RGB), 25, 224, 224])
        # print(l)                   tensor([16])
        # ---------------------- RGB EXTRACTION ----------------------
        output_rgb, feat_rgb = model_rgb(rgb_input)
        feat_rgb = feat_rgb["features"]
        sample_rgb = feat_rgb[0]      # torch.Size([1, 1024])
        embeddings_rgb.append(sample_rgb)

        # ---------------------- EMG EXTRACTION ----------------------
        outputs_emg, feat_emg = model_emg(emg_input)
        sample_emg = feat_emg[0]      # torch.Size([64])
        embeddings_emg.append(sample_emg)

        # ---------------------- EMG STAT EXTRACTION ----------------------
        #embeddings = EMG_Feature_Extractor(emg_input[0])
        #embeddings_emg.append(embeddings)

print(len(embeddings_emg))
print(len(embeddings_rgb))

features_emg = "saved_features/FINE_TUNING_emg_s04.pkl"
features_rgb = "saved_features/FINE_TUNING_rgb_s04.pkl"

with open(features_rgb, 'wb') as f:
    pickle.dump(embeddings_rgb, f)

with open(features_emg, 'wb') as f:
    pickle.dump(embeddings_emg, f)


186
186


### SETUP

In [50]:
BATCH_SIZE = 16
EPOCHS = 100
LR = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 50
GAMMA = 0.1

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("------ USING APPLE SILICON GPU ------")


In [51]:
import pandas as pd
from utils.loaders import FeaturesTuningDataset

rgb = FeaturesTuningDataset("./saved_features/FINE_TUNING_rgb_s04.pkl")
emg = FeaturesTuningDataset("./saved_features/FINE_TUNING_emg_s04.pkl")
print(f'RGB dataset size: {len(rgb)}')

loader_rgb = DataLoader(rgb, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=False, pin_memory=False)
loader_emg = DataLoader(emg, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=False, pin_memory=False)

rgb_t = FeaturesTuningDataset("./saved_features/FINE_TUNING_rgb_s04_TEST.pkl")
emg_t = FeaturesTuningDataset("./saved_features/FINE_TUNING_emg_s04_TEST.pkl")

loader_rgb_t = DataLoader(rgb_t, batch_size=1, shuffle=False, num_workers=0, drop_last=False, pin_memory=False)
loader_emg_t = DataLoader(emg_t, batch_size=1, shuffle=False, num_workers=0, drop_last=False, pin_memory=False)

# print device loaders
for (rgb_batch_idx, x) in enumerate(loader_rgb):
    print(x.device)  # result --> cpu
    break

RGB dataset size: 19
cpu


### TRAINING

In [53]:
from train_vae import loss_function
from utils.logger import setup_logger
from torch.autograd import Variable
from torch.optim import Adam
from train_vae import train_tuning, evaluate_tuning

# LSTM EMG dimension is 64 
# STAT EMG dimension is 112
model_finetune = FC_VAE(dim_input=1024, nz=64, dim_output=1024, n_hidden=512, device=DEVICE)
model_finetune.to(DEVICE)

# Carica i pesi del modello RGB per l'encoder
checkpoint_rgb = torch.load('./saved_models/VAE_RGB/final_VAE_RGB_epoch_50.pth', map_location=DEVICE)
# Rimuovi il prefisso 'encoder.' dalle chiavi dello state_dict
checkpoint_rgb = {k.replace('encoder.', ''): v for k, v in checkpoint_rgb.items() if 'encoder' in k}
model_finetune.encoder.load_state_dict(checkpoint_rgb)

# Carica i pesi del modello EMG per il decoder
checkpoint_emg = torch.load('./saved_models/VAE_EMG/final_VAE_EMG_epoch_50.pth', map_location=DEVICE)
checkpoint_emg = {k.replace('decoder.', ''): v for k, v in checkpoint_emg.items() if 'decoder' in k}
model_finetune.decoder.load_state_dict(checkpoint_emg)

# Create Optimizer & Scheduler objects
optimizer = Adam(model_finetune.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

train_tuning(model_finetune, optimizer, EPOCHS, DEVICE, loader_rgb, loader_emg, BATCH_SIZE, scheduler)
evaluate_tuning(model_finetune, DEVICE, loader_rgb_t, loader_emg_t)

torch.save(model_finetune.state_dict(), f'./saved_models/VAE_Fine_Tuninng/VAE_RGB_to_EMG_LSTM_epoch_{EPOCHS}.pth')


2024-06-12 21:57:02 LOG INFO 	Epoch, 1, 	Average Loss: , 0.00885642478665845
2024-06-12 21:57:02 LOG INFO 	Epoch, 2, 	Average Loss: , 0.006324561566791751
2024-06-12 21:57:02 LOG INFO 	Epoch, 3, 	Average Loss: , 0.006045587604272772
2024-06-12 21:57:02 LOG INFO 	Epoch, 4, 	Average Loss: , 0.005697414252526042
2024-06-12 21:57:03 LOG INFO 	Epoch, 5, 	Average Loss: , 0.005372994505292313
2024-06-12 21:57:03 LOG INFO 	Epoch, 6, 	Average Loss: , 0.004979134106013755
2024-06-12 21:57:04 LOG INFO 	Epoch, 7, 	Average Loss: , 0.0046380482487041845
2024-06-12 21:57:04 LOG INFO 	Epoch, 8, 	Average Loss: , 0.004268210196063261
2024-06-12 21:57:05 LOG INFO 	Epoch, 9, 	Average Loss: , 0.004011058285620741
2024-06-12 21:57:05 LOG INFO 	Epoch, 10, 	Average Loss: , 0.003699524021199481
2024-06-12 21:57:05 LOG INFO 	Epoch, 11, 	Average Loss: , 0.0030873848326419566
2024-06-12 21:57:06 LOG INFO 	Epoch, 12, 	Average Loss: , 0.002783575058195063
2024-06-12 21:57:06 LOG INFO 	Epoch, 13, 	Average Loss: , 0.

Test Loss: 0.0744
